# Загрузка и первичный анализ данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

from urllib.parse import urlencode

In [ ]:
RANDOM_STATE = 42

In [ ]:
TEST = "https://www.dropbox.com/s/asf4b1z1yme5o7u/cars_test.csv?dl=1"
TRAIN = "https://www.dropbox.com/s/qk4b79i7c078sxm/cars_train.csv?dl=1"

In [ ]:
data = pd.read_csv(TRAIN)

In [ ]:
data.shape

Выделим целевую переменную `sellingprice` в отдельную переменную `y`, а `X` - матрица объект-признак.

In [ ]:
X = data.drop('sellingprice', axis=1)
y = data['sellingprice']

In [ ]:
X.head()

Посмотрим на типы колонок и число пропущенных значений в них.

In [ ]:
X.info()

Посмотрим на числовые признаки

In [ ]:
X.describe()

Посмотрим на категориальные признаки

In [ ]:
X.describe(include='object')

Признак `vin` это уникальный идентификатор машины, поэтому удалим его.

In [ ]:
X.drop('vin', axis=1, inplace=True)

# Разведочный анализ данных

Заполним пропуски в числовых столбцах средним значением, а в категориальных - пустой категорией

In [ ]:
for i in X.columns:
    if X[i].dtype == 'object':
          X[i].fillna("", inplace=True)

In [ ]:
for i in X.columns:
    if X[i].dtype != 'object':
          mean = np.mean(X[i])
          X[i].fillna(mean, inplace=True)

## Обработка категориальных признаков

Посмотрим на количество значений в каждой категории

In [ ]:
for c in X.columns:
    if X[c].dtype == 'object':
          print(c, len(X[c].unique()))

В saledate очень много различных значений. Обработаем saledate:

In [ ]:
X['car_age'] = X['saledate'].apply(lambda x: int(x.split(" ")[3])) - X['year']
X['date'] = X['saledate'].apply(lambda x: x.split(" ")[1]+x.split(" ")[3])

X.drop('saledate', axis=1, inplace=True)

## Корреляционные матрицы

In [ ]:
import seaborn as sb

X['target'] = y

cols = X.columns[X.dtypes != 'object']

corr = X[cols].corr()
sb.heatmap(corr, cmap="Blues", annot=True)

X.drop('target', axis=1, inplace=True)

Все числовые признаки важны. Посмотрим на аналог корреляции категориальных признаков

In [ ]:
import association_metrics as am

XC = X.apply(
        lambda x: x.astype("category") if x.dtype == "object" else x)

cramersv = am.CramersV(XC)

cramersv.fit()

Признаки make и model сильно связаны, поэтому уберем make как менее информативный

In [ ]:
X.drop('make', axis=1, inplace=True)

Посмотрим влияние категориального признака на целевую переменную

In [ ]:
import seaborn as sns

sns.scatterplot(data=X.iloc[:100], x=X.iloc[:100].index, y=y[:100], hue='date');

In [ ]:
sns.scatterplot(data=X.iloc[:100], x=X.iloc[:100].index, y=y[:100], hue='interior');

In [ ]:
sns.scatterplot(data=X.iloc[:100], x=X.iloc[:100].index, y=y[:100], hue='transmission');

Посмотрим на распределение целевой переменной

In [ ]:
plt.hist(y, bins=100);

## Поиск аномальных значений

In [ ]:
cat_cols = X.columns[X.dtypes == 'object']
num_cols = X.columns[X.dtypes != 'object']

In [ ]:
for col in num_cols:
    print(col)
    sb.boxplot(X[col])
    plt.show();

In [ ]:
X[X['odometer'] > 800000][['car_age','odometer']]

Выкинем машины младше 10 лет, проехавшие 1000000 миль - это почти точно выбросы. Удалим car_age, так как в дате есть столбец yaer

In [ ]:
Xnew = X[~((X.car_age < 10) & (X.odometer > 800_000))]
ynew = y[~((X.car_age < 10) & (X.odometer > 800_000))]

Xnew.drop('car_age', axis=1, inplace=True);
print(Xnew)

# Построение baseline-модели

Для baseline-модели:
1) закодируем категориальные признаки при помощи TargetEncoder
2) масштабируем все признаки с помощью StandardScaler
3) обучим линейную регрессию

In [ ]:
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xnew, ynew, test_size=0.25, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


enc = TargetEncoder(cols=cat_cols)
enc.fit(X_train, y_train)
X_train_new = enc.transform(X_train)
X_test_new = enc.transform(X_test)

scaler = StandardScaler()
scaler.fit(X_train_new)
X_train_new = pd.DataFrame(scaler.transform(X_train_new), columns=X_train.columns)
X_test_new = pd.DataFrame(scaler.transform(X_test_new), columns=X_test.columns)

model = LinearRegression()
model.fit(X_train_new, y_train)
pred = model.predict(X_test_new)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as MAPE

MAPE(y_test, pred)

# Pipelines

In [ ]:
from sklearn.pipeline import Pipeline

p1 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', LinearRegression())
    ])
p2 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=1)),
    ('scaler_', StandardScaler()),
    ('model_', LinearRegression())
    ])

p3 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=100)),
    ('scaler_', StandardScaler()),
    ('model_', LinearRegression())
    ])

p4 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=1)),
    ('scaler_', MinMaxScaler()),
    ('model_', LinearRegression())
    ])

p5 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=10)),
    ('scaler_', MinMaxScaler()),
    ('model_', LinearRegression())
    ])

p6 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=100)),
    ('scaler_', MinMaxScaler()),
    ('model_', LinearRegression())
    ])

p7 = Pipeline([
    ('encoder_',LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', LinearRegression())
    ])

p8 = Pipeline([
    ('encoder_',LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', MinMaxScaler()),
    ('model_', LinearRegression())
    ])

In [ ]:
for i,p in enumerate([p1,p2,p3,p4,p5,p6,p7,p8]):
    p.fit(X_train, y_train)
    pred = p.predict(X_test)
    print(i+1, MAPE(y_test, pred))

Кодировка и масштабирование не улучшили модель, сменим модель

In [ ]:
p9 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', RandomForestRegressor(n_jobs=-1))
    ])

p9.fit(X_train, y_train)
pred = p9.predict(X_test)
MAPE(y_test, pred)

Попробуем для RandomForest поменять кодировщик, его гиперпараметры и скалер.

In [ ]:
p10 = Pipeline([
    ('encoder_',LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', RandomForestRegressor(n_jobs=-1))
    ])

p11 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols)),
    ('scaler_', MinMaxScaler()),
    ('model_', RandomForestRegressor(n_jobs=-1))
    ])

p12 = Pipeline([
    ('encoder_',LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', MinMaxScaler()),
    ('model_', RandomForestRegressor(n_jobs=-1))
    ])

p13 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=1)),
    ('scaler_', StandardScaler()),
    ('model_', RandomForestRegressor(n_jobs=-1))
    ])

In [ ]:
for i,p in enumerate([p9,p10,p11,p12,p13]):
    p.fit(X_train.iloc[:50000], y_train[:50000])
    pred = p.predict(X_test)
    print(i+9, MAPE(y_test, pred))

In [ ]:
p9.fit(X_train, y_train)
pred = p9.predict(X_test)
MAPE(y_test, pred)

Посмотрим на важность признаков

In [ ]:
weights = pd.DataFrame(p9['model_'].feature_importances_, index=X_train.columns).sort_values(by=0, ascending=False)
weights

# Попытки улучшить модель

In [ ]:
plt.hist(y, bins=100);

Попробуем сделать распределение целевой переменной более похожим на нормальное и заново обучить лучшую модель с предыдущего шага.

In [ ]:
plt.hist(np.log(y), bins=100);

In [ ]:
y_train = np.log(y_train)
y_test = np.log(y_test)

In [ ]:
p9 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', RandomForestRegressor(n_jobs=-1))
    ])

p9.fit(X_train, y_train)
pred = p9.predict(X_test)

MAPE(np.exp(y_test), np.exp(pred))

Преобразование целевой переменной улучшило качество модели

# Optuna

In [ ]:
data_pipeline = Pipeline([
        ('encoder_',TargetEncoder(cols=cat_cols)),
        ('scaler_', StandardScaler())
])

X_train_good = data_pipeline.fit_transform(X_train, y_train)
X_test_good = data_pipeline.transform(X_test)

In [ ]:
import optuna

def objective(trial):

    param = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "max_features": trial.suggest_categorical("max_features", ['sqrt', 'log2', None])
    }

    estimator = RandomForestRegressor(**param, verbose=False, n_jobs=-1)

    estimator.fit(X_train_good[:50000], y_train.iloc[:50000])
    pred = estimator.predict(X_test_good)

    return MAPE(np.exp(y_test), np.exp(pred))

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)
print(study.best_trial)

In [ ]:
model = RandomForestRegressor(n_estimators=745, n_jobs=-1)

model.fit(X_train_good, y_train)
pred = model.predict(X_test_good)
MAPE(np.exp(y_test), np.exp(pred))

Лучшая модель сейчас RandomForest

Попробуем CatBoost

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor()

model.fit(X_train_good, y_train)
pred = model.predict(X_test_good)

MAPE(np.exp(y_test), np.exp(pred))

Подберем гиперпараметры буста

In [ ]:
import optuna

def objective(trial):

    param = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        "max_depth": trial.suggest_int("max_depth", 2, 16),
    }

    estimator = CatBoostRegressor(**param, verbose=False)

    estimator.fit(X_train_good[:50000], y_train.iloc[:50000])
    pred = estimator.predict(X_test_good)

    return MAPE(np.exp(y_test), np.exp(pred))

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)
print(study.best_trial)

In [ ]:
model = CatBoostRegressor(n_estimators=477, max_depth=11)

model.fit(X_train_good, y_train)
pred_cb = model.predict(X_test_good)

MAPE(np.exp(y_test), np.exp(pred_cb))

Пусть CatBoost сам закодирует признаки

In [ ]:
cat_features = [1,2,3,4,5,8,9,10,11]

model = CatBoostRegressor(cat_features=cat_features)

model.fit(X_train, y_train)
pred = model.predict(X_test)

MAPE(np.exp(y_test), np.exp(pred))

# Stacking и Blending

### Простое смешивание

* RandomForest - 0.178
* CatBoost - 0.170

In [ ]:
pred_final = 0.3 * pred + 0.7 * pred_cb

MAPE(np.exp(y_test), np.exp(pred_final))

### Stacking и blending

In [ ]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('rf', RandomForestRegressor(n_jobs=-1)),
    ('cb', CatBoostRegressor(n_estimators=477, max_depth=11))
    ]

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10, random_state=42))

In [ ]:
reg.fit(X_train_good, y_train)
pred_stacking = reg.predict(X_test_good)

MAPE(np.exp(y_test), np.exp(pred_stacking))

# Сохранение результатов

In [ ]:
test_data = pd.read_csv(TEST)

Заполняем пропуски

In [ ]:
for c in test_data.columns:
    if test_data[c].dtype == 'object':
          test_data[c].fillna("", inplace=True)

In [ ]:
for c in test_data.columns:
    if test_data[c].dtype != 'object':
          mean = np.mean(X[c])
          test_data[c].fillna(mean, inplace=True)

Обрабатываем дату

In [ ]:
test_data['date'] = test_data['saledate'].apply(lambda x: x.split(" ")[1]+x.split(" ")[3])
test_data.drop(['vin','make','saledate'], axis=1, inplace=True)

In [ ]:
test_good = data_pipeline.transform(test_data)
test_pred = reg.predict(test_good)

Сохраняем предсказания в файл

In [ ]:
test_data['prediction'] = test_pred

test_data[['prediction']].to_csv("test_prediction.csv", index=False)

### Сохраним модель

In [ ]:
import pickle

with open('model.pickle', 'wb') as f:
    pickle.dump(model, f)
    
'''
 А так модель можно загрузить из файла:
with open('filename.pickle', 'rb') as f:
    model = pickle.load(f)
'''